In [1]:
import os
os.chdir('/workspaces/work_utils2')

In [2]:
import presentation

In [3]:
ppt = presentation.PowerPointPresentation(template_filepath='presentation/assets/Blank.pptx')


In [4]:
slide = ppt.add_slide('Title Only')
slide.set_title(title='Title', subtitle='Subtitle')

KeyError: 'italic'

In [ ]:
ppt.save('test.pptx')